In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
import xgboost as xgb




In [2]:
train=pd.read_csv('/kaggle/input/liver-guard-multi-class-prediction-for-cirrhosis/train.csv')
test=pd.read_csv('/kaggle/input/liver-guard-multi-class-prediction-for-cirrhosis/test.csv')

In [3]:
train['Age'] = train['Age'] // 365.0

In [4]:
train['N_Days']=train['N_Days']//365.0

In [5]:
# Rename column "old_name" to "new_name"
train.rename(columns={"N_Days": "Years"})

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Years,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,3.0,NaN,67.0,F,NaN,NaN,NaN,N,0.8,NaN,3.65,NaN,NaN,NaN,NaN,331.0,11.2,4.0,D
1,1,8.0,NaN,46.0,F,NaN,NaN,NaN,N,0.8,NaN,3.87,NaN,NaN,NaN,NaN,269.0,10.3,1.0,C
2,2,12.0,NaN,46.0,F,NaN,NaN,NaN,S,1.3,NaN,3.40,NaN,NaN,NaN,NaN,219.0,11.5,4.0,D
3,3,2.0,D-penicillamine,39.0,F,N,Y,Y,N,1.2,244.0,3.08,121.0,11046.6,79.05,91.0,358.0,11.0,4.0,C
4,4,0.0,NaN,61.0,F,NaN,NaN,NaN,N,2.7,NaN,3.91,NaN,NaN,NaN,NaN,142.0,11.5,4.0,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,12.0,Placebo,64.0,F,N,N,N,N,0.5,248.0,3.70,48.0,678.0,51.15,84.0,344.0,9.7,3.0,C
14996,14996,3.0,D-penicillamine,47.0,M,N,Y,N,N,1.6,660.0,2.95,94.0,1857.0,151.90,155.0,337.0,11.7,4.0,D
14997,14997,7.0,Placebo,49.0,F,N,Y,Y,N,0.7,252.0,3.85,13.0,663.0,72.85,58.0,336.0,9.6,3.0,C
14998,14998,3.0,NaN,64.0,F,NaN,NaN,NaN,N,0.5,NaN,3.49,NaN,NaN,NaN,NaN,425.0,10.4,4.0,C


In [6]:
TARGET = "Status"
FEATURES = train.columns.drop(["id", "N_Days", TARGET]).tolist()

# Separate numerical and categorical features
numerical_cols = train[FEATURES].select_dtypes(include="number").columns.tolist()
categorical_cols = train[FEATURES].select_dtypes(include="object").columns.tolist()

# Fill missing values
train[numerical_cols] = train[numerical_cols].fillna(0)
test[numerical_cols] = test[numerical_cols].fillna(0)
train[categorical_cols] = train[categorical_cols].fillna("NV")
test[categorical_cols] = test[categorical_cols].fillna("NV")

# Filter valid categorical columns (must have >1 unique value)
valid_categorical = [col for col in categorical_cols if train[col].nunique() > 1]

# One-hot encode
ohe = OneHotEncoder(drop="first", sparse_output=False)  # <-- Fix here
X_cat_train = train[valid_categorical]
X_cat_test = test[valid_categorical]
train_ohe = ohe.fit_transform(X_cat_train)
test_ohe = ohe.transform(X_cat_test)

# Get encoded column names
encoded_columns = ohe.get_feature_names_out(valid_categorical)

# Combine numerical and encoded categorical features
X_train = pd.concat([
    train[numerical_cols],
    pd.DataFrame(train_ohe, columns=encoded_columns)
], axis=1)

X_test = pd.concat([
    test[numerical_cols],
    pd.DataFrame(test_ohe, columns=encoded_columns)
], axis=1)

In [7]:
print("Training shape:", X_train.shape)
print("Test shape:", X_test.shape)


Training shape: (15000, 22)
Test shape: (10000, 22)


In [8]:
# Encode target variable (Status)
le = LabelEncoder()
y_train = le.fit_transform(train["Status"])  # <-- Define y_train here

# Preprocess features (use your existing code for X_train and X_test)

# Train XGBoost
model = xgb.XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    eval_metric="mlogloss",
    random_state=42
)
model.fit(X_train, y_train)  # Now y_train is defined

# Predict and save submission (use your existing code)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=3, num_parallel_tree=None, ...)

In [9]:
print(y_train[:5])  # Should output array like [0, 1, 2, 0, ...]
print("Class mapping:", le.classes_)  # Should be ['C', 'CL', 'D']

[2 0 2 0 2]
Class mapping: ['C' 'CL' 'D']


In [10]:
# Load preprocessed data (X_train, X_test from earlier steps)
# Encode target variable
le = LabelEncoder()
y_train = le.fit_transform(train["Status"])  # Already done

# Verify class mapping
print("Class mapping:", le.classes_)  # Should output ['C', 'CL', 'D']

# Train XGBoost
model = xgb.XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    eval_metric="mlogloss",
    random_state=42
)
model.fit(X_train, y_train)

# Predict probabilities (ensure alignment with le.classes_)
probabilities = model.predict_proba(X_test)
probabilities = np.clip(probabilities, 1e-15, 1 - 1e-15)

# Create submission DataFrame
submission = pd.DataFrame({
    "id": test["id"],
    "Status_C": probabilities[:, 0],  # Class 0: C
    "Status_CL": probabilities[:, 1], # Class 1: CL
    "Status_D": probabilities[:, 2]   # Class 2: D
})

# Save to CSV
submission.to_csv("submission.csv", index=False)

Class mapping: ['C' 'CL' 'D']


In [11]:
from sklearn.model_selection import train_test_split
# Split training data into 80% train, 20% validation (stratified by `Status`)
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    stratify=y_train,
    random_state=42
)

In [12]:
# Predict classes (C/CL/D)
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.9500


In [13]:
# Predict probabilities for all classes
y_proba = model.predict_proba(X_val)
logloss = log_loss(y_val, y_proba)
print(f"Validation Log Loss: {logloss:.4f}")

Validation Log Loss: 0.1659


In [14]:
# Convert encoded labels back to original names
class_names = le.classes_  # ['C', 'CL', 'D']
print(classification_report(
    y_val,
    y_pred,
    target_names=class_names
))

              precision    recall  f1-score   support

           C       0.95      0.98      0.96      2031
          CL       1.00      0.78      0.88        68
           D       0.95      0.89      0.92       901

    accuracy                           0.95      3000
   macro avg       0.97      0.88      0.92      3000
weighted avg       0.95      0.95      0.95      3000



In [15]:
print(submission.head())

      id  Status_C  Status_CL  Status_D
0  15000  0.769844   0.019195  0.210961
1  15001  0.994765   0.000040  0.005195
2  15002  0.922970   0.000948  0.076083
3  15003  0.987983   0.000216  0.011801
4  15004  0.984142   0.000069  0.015790


In [16]:
print(submission.shape)

(10000, 4)
